<a href="https://colab.research.google.com/github/JKashyap46/ai5-VQA/blob/main/Visual_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
import json

import operator

import pandas as pd
import progressbar

from sklearn.preprocessing import LabelEncoder
import pickle as pk
from collections import defaultdict
from itertools import zip_longest
import numpy as np
from keras.utils import np_utils

import spacy

import tensorflow as tf
import keras

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'ai5-c1-group3'
bucket_name = 'vqa-dataset-bucket'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
# !gsutil cp gs://{bucket_name}/v2_Questions_Val_mscoco.zip ./vqa-dataset-bucket/v2_Questions_Val_mscoco.zip
# !gsutil cp gs://{bucket_name}/v2_Annotations_Val_mscoco.zip ./vqa-dataset-bucket/v2_Annotations_Val_mscoco.zip
!gsutil cp gs://{bucket_name}/val2014.zip ./vqa-dataset-bucket/val2014.zip

Copying gs://vqa-dataset-bucket/val2014.zip...
/ [1 files][  6.2 GiB/  6.2 GiB]   49.3 MiB/s                                   
Operation completed over 1 objects/6.2 GiB.                                      


In [ ]:
# with zipfile.ZipFile(os.path.join('/content/vqa-dataset-bucket/','v2_Questions_Val_mscoco.zip')) as zfile:
#   zfile.extractall('/content/vqa-dataset-bucket/')

# with zipfile.ZipFile(os.path.join('/content/vqa-dataset-bucket/','v2_Annotations_Val_mscoco.zip')) as zfile:
#   zfile.extractall('/content/vqa-dataset-bucket/')

with zipfile.ZipFile(os.path.join('/content/vqa-dataset-bucket/','val2014.zip')) as zfile:
  zfile.extractall('/content/vqa-dataset-bucket/')

In [ ]:
# !gsutil cp  /content/vqa-dataset-bucket/v2_OpenEnded_mscoco_val2014_questions.json gs://{bucket_name}/v2_OpenEnded_mscoco_val2014_questions.json
# !gsutil cp  /content/vqa-dataset-bucket/v2_mscoco_val2014_annotations.json gs://{bucket_name}/v2_mscoco_val2014_annotations.json
# !gsutil -m cp -r  /content/vqa-dataset-bucket/val2014 gs://{bucket_name}/val2014


BucketNotFoundException: 404 gs://{bucket_name} bucket does not exist.


In [ ]:
!gsutil cp gs://{bucket_name}/v2_OpenEnded_mscoco_val2014_questions.json ./vqa-dataset-bucket/v2_OpenEnded_mscoco_val2014_questions.json
!gsutil cp gs://{bucket_name}/v2_mscoco_val2014_annotations.json ./vqa-dataset-bucket/v2_mscoco_val2014_annotations.json

Copying gs://vqa-dataset-bucket/v2_OpenEnded_mscoco_val2014_questions.json...
/ [1 files][ 19.3 MiB/ 19.3 MiB]                                                
Operation completed over 1 objects/19.3 MiB.                                     
Copying gs://vqa-dataset-bucket/v2_mscoco_val2014_annotations.json...
| [1 files][163.8 MiB/163.8 MiB]                                                
Operation completed over 1 objects/163.8 MiB.                                    


In [ ]:
questions = json.load(open('./vqa-dataset-bucket/v2_OpenEnded_mscoco_val2014_questions.json'))
annotation = json.load(open('./vqa-dataset-bucket/v2_mscoco_val2014_annotations.json'))
annotations = annotation['annotations']

In [ ]:
annotations_df = pd.DataFrame(annotations)
annotations_df.head()

,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id
0,none of the above,down,"[{'answer': 'down', 'answer_confidence': 'yes'...",262148,other,262148000
1,what are the,watching,"[{'answer': 'spectating', 'answer_confidence':...",262148,other,262148001
2,what is,picnic table,"[{'answer': 'table', 'answer_confidence': 'yes...",262148,other,262148002
3,what,foodiebakercom,"[{'answer': 'foodiebakercom', 'answer_confiden...",393225,other,393225000
4,is this a,no,"[{'answer': 'no', 'answer_confidence': 'yes', ...",393225,yes/no,393225001


In [ ]:
annotations_df.shape

(214354, 6)

In [ ]:
total_questions = len(questions['questions'])

In [ ]:
training_questions = questions['questions'][0:50000]
train_questions = []
progress = progressbar.ProgressBar()
for index in progress(range(50000)):
  train_questions.append(training_questions[index]['question'])

100% (50000 of 50000) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


In [ ]:
questions_df = pd.DataFrame(training_questions)

In [ ]:
questions_df.head()

,image_id,question,question_id
0,262148,Where is he looking?,262148000
1,262148,What are the people in the background doing?,262148001
2,262148,What is he on top of?,262148002
3,393225,What website copyrighted the picture?,393225000
4,393225,Is this a creamy soup?,393225001


In [ ]:
def most_freq_answer(values):
    ans_dict = {}
    for index in range(10):
        ans_dict[values[index]['answer']] = 1
    for index in range(10):
        ans_dict[values[index]['answer']] += 1

    return max(ans_dict.items(), key = operator.itemgetter(1))[0]

In [ ]:
progress = progressbar.ProgressBar()
answer_train = []
for index in progress(range(50000)):
  answer_train.append(most_freq_answer(annotations[index]['answers']))

100% (50000 of 50000) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


In [ ]:
len(answer_train)

50000

In [ ]:
images_train = []
progress = progressbar.ProgressBar()
for index in progress(range(50000)):
  images_train.append(training_questions[index]['image_id'])

100% (50000 of 50000) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


In [ ]:
def freq_answers(training_questions, answer_train, images_train, upper_lim):

    freq_ans = defaultdict(int)
    for ans in answer_train:
        freq_ans[ans] += 1

    sort_freq = sorted(freq_ans.items(), key=operator.itemgetter(1), reverse=True)[0:upper_lim]
    top_ans, top_freq = zip(*sort_freq)
    new_answers_train = list()
    new_questions_train = list()
    new_images_train = list()
    for ans, ques, img in zip(answer_train, training_questions, images_train):
        if ans in top_ans:
            new_answers_train.append(ans)
            new_questions_train.append(ques)
            new_images_train.append(img)

    return (new_questions_train, new_answers_train, new_images_train, top_ans)

In [ ]:
upper_lim = 1000 #Number of most frequently occurring answers in COCOVQA (85%+)
training_questions, answers_train, images_train,top_ans = freq_answers(train_questions, answer_train, images_train, upper_lim)
print (len(training_questions), len(answers_train),len(images_train))

39177 39177 39177


In [ ]:
lbl = LabelEncoder()
lbl.fit(answers_train)
nb_classes = len(list(lbl.classes_))
pk.dump(lbl, open('./label_encoder.sav','wb'))

In [ ]:
nb_classes

1000

In [ ]:
def get_questions_sum(questions, nlp):
    nb_samples = len(questions)
    word2vec_dim = nlp(questions[0])[0].vector.shape[0]
    ques_matrix = np.zeros((nb_samples, word2vec_dim))
    for index in range(len(questions)):
        tokens = nlp(questions[index])
        for j in range(len(tokens)):
            ques_matrix[index,:] += tokens[j].vector

    return ques_matrix

In [ ]:
# !gsutil cp gs://{bucket_name}/GoogleNews-vectors-negative300.bin.gz ./vqa-dataset-bucket/GoogleNews-vectors-negative300.bin.gz

Copying gs://vqa-dataset-bucket/GoogleNews-vectors-negative300.bin.gz...
| [1 files][  1.5 GiB/  1.5 GiB]   31.8 MiB/s                                   
Operation completed over 1 objects/1.5 GiB.                                      


In [ ]:
import gensim
import spacy

# Path to google news vectors
google_news_path = "./vqa-dataset-bucket/GoogleNews-vectors-negative300.bin.gz"

# Load google news vecs in gensim
model = gensim.models.KeyedVectors.load_word2vec_format(google_news_path, binary=True)

In [ ]:
nlp = spacy.blank('en')

# Loop through range of all indexes, get words associated with each index.
# The words in the keys list will correspond to the order of the google embed matrix
keys = model.vocab.keys()

# Set the vectors for our nlp object to the google news vectors
nlp.vocab.vectors = spacy.vocab.Vectors(data=model.syn0, keys=model.index2word)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [ ]:
questions_matrix = get_questions_sum(training_questions, nlp)

In [ ]:
questions_matrix[0].shape

(300,)

In [ ]:
pk.dump(questions_matrix, open('./questions_mat_50k','wb'))

In [ ]:
# !gsutil cp  ./questions_mat_50k gs://{bucket_name}/questions_mat_50k
# !gsutil cp  gs://{bucket_name}/questions_mat ./questions_mat


BucketNotFoundException: 404 gs://{bucket_name} bucket does not exist.


In [ ]:
# del nlp

In [ ]:
del questions
del annotation
del annotations
del progress

In [ ]:
questions_matrix = pk.load(open('./questions_mat_50k','rb'))

In [ ]:
def get_answers_sum(answers, encoder):
    y = encoder.transform(answers)

    nb_classes = encoder.classes_.shape[0]
    Y = np_utils.to_categorical(y, nb_classes)
    del y
    del nb_classes
    return Y

In [ ]:
encoded_answers = get_answers_sum(answers_train, lbl)

In [ ]:
# pk.dump(encoded_answers, open('./encoded_ans_50k','wb'))

In [ ]:
# !gsutil cp  ./encoded_ans_50k gs://{bucket_name}/encoded_ans_50k
#!gsutil cp  gs://{bucket_name}/encoded_ans ./encoded_ans


BucketNotFoundException: 404 gs://{bucket_name} bucket does not exist.


In [ ]:
# encoded_answers = pk.load(open('./encoded_ans','rb'))


In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

In [ ]:
def get_image_data(img):
  img = tf.io.read_file(img)
  resized_image_data = decode_img(img)
  return resized_image_data

In [ ]:
def get_tensors(text_input, image_input, y):
  return (image_input, text_input), y


def create_dataset(question_matrix,image_input, encoded_answers):
  question_dataset = tf.convert_to_tensor(question_matrix)
  question_dataset = tf.data.Dataset.from_tensor_slices(question_dataset)

  images = tf.data.Dataset.list_files(image_input)
  image_tensor = images.map(get_image_data)
  y_dataset = tf.convert_to_tensor(encoded_answers)
  y_dataset = tf.data.Dataset.from_tensor_slices(y_dataset)
  dataset = tf.data.Dataset.zip((question_dataset, image_tensor, y_dataset))
  dataset = dataset.shuffle(1000)
  dataset = dataset.batch(100)
  dataset = dataset.map(get_tensors)
  dataset = dataset.prefetch(3)
  return dataset

In [ ]:
image_train = [os.path.join('/content/vqa-dataset-bucket/val2014','COCO_val2014_'+str(image).zfill(12)+'.jpg') for image in images_train]

In [ ]:
train_dataset = create_dataset(questions_matrix, image_train, encoded_answers)

In [ ]:
# Import Keras 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, Sequential

# Define CNN for Image Input
vision_model = Sequential()
vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
vision_model.add(Conv2D(64, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(128, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Flatten())

image_input = Input(shape=(224, 224, 3))
encoded_image = vision_model(image_input)

# Define RNN for language input
question_input=  tf.keras.layers.Input(shape=(None,))
embedded_question= tf.keras.layers.Embedding(300, output_dim=50)(question_input)
encoded_question= tf.keras.layers.LSTM(100)(embedded_question)

# Combine CNN and RNN to create the final model
merged = keras.layers.concatenate([encoded_question, encoded_image])
dense_layer = Dense(2000, activation='relu')(merged)
output = Dense(1000, activation='softmax')(dense_layer)
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

In [ ]:
vqa_model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     15000       input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 100)          60400       embedding[0][0]                  
______________________________________________________________________________________________

In [ ]:
vqa_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = vqa_model.fit(train_dataset, epochs=10, steps_per_epoch=50)

Epoch 1/10
50/50 [==============================] - 42s 527ms/step - loss: 26.9474 - accuracy: 0.1944
Epoch 2/10
50/50 [==============================] - 26s 527ms/step - loss: 4.4367 - accuracy: 0.1994
Epoch 3/10
50/50 [==============================] - 26s 527ms/step - loss: 4.2860 - accuracy: 0.2192
Epoch 4/10
50/50 [==============================] - 26s 527ms/step - loss: 4.2614 - accuracy: 0.2202
Epoch 5/10
50/50 [==============================] - 26s 528ms/step - loss: 4.2306 - accuracy: 0.2126
Epoch 6/10
50/50 [==============================] - 26s 528ms/step - loss: 4.2782 - accuracy: 0.2158
Epoch 7/10
50/50 [==============================] - 26s 527ms/step - loss: 4.2354 - accuracy: 0.2110
Epoch 8/10
50/50 [==============================] - 26s 527ms/step - loss: 4.2990 - accuracy: 0.2190
Epoch 9/10
50/50 [==============================] - 26s 521ms/step - loss: 4.2686 - accuracy: 0.2132


In [ ]:
# vqa_model.save("vqa_model_word2vec_lstm_50k")

INFO:tensorflow:Assets written to: vqa_model_word2vec_lstm_50k/assets


INFO:tensorflow:Assets written to: vqa_model_word2vec_lstm_50k/assets


In [ ]:
# !gsutil -m cp -r ./vqa_model_word2vec_lstm gs://{bucket_name}/vqa_model_word2vec_lstm

Copying file://./vqa_model_word2vec_lstm/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./vqa_model_word2vec_lstm/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://./vqa_model_word2vec_lstm/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./vqa_model_word2vec_lstm/variables/variables.index [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crc

In [ ]:
filename = 'hello'